In [1]:
import numpy as np
import torch
from tqdm import tqdm
import os


In [24]:
'''
choose between train and val
'''
# filename = 'val'
filename = 'train'

In [13]:
def f(a,b):
    assert len(a) == len(b)
    return [a[b[i]-1] for i in range(len(b))]

def permute(list_of_permutations):
    answer = list(range(1,len(list_of_permutations[0])+1))
    for p in list_of_permutations:
        answer = f(answer, list(p))
    return answer

In [25]:
rng = np.random.default_rng(1234)
block_size = 256
p = 5
C = block_size // (2 * (p+2))
N = 1000000
random_list = rng.permuted(np.tile(np.arange(1,p+1), N * C).reshape(N*C,p), axis=1).reshape(N,C,p)
input_length_list = np.random.randint(low = 1,high =C+1, size = (N,), dtype = np.int8)
token_id_list = []
for i in range(N):
    current_id = []
    current_input_tokens = random_list[i]
    for j in range(C):
        current_id += [-3]+ list(current_input_tokens[j])+ [-4]
    token_id_list.append(current_id + [-2] + [-3]+ list(permute(random_list[i]))+ [-4])

In [44]:
token_id_list[0][:-1]

[5, -2]

In [26]:
dtype = np.int8
arr = np.memmap(filename + '.bin', dtype=dtype, mode='w+', shape=(block_size * len(token_id_list)))
arr_label = np.memmap(filename + '_label.bin', dtype=dtype, mode='w+', shape=(block_size * len(token_id_list)))
print(f"writing {filename}...")
for jj, example in tqdm(enumerate(token_id_list)):
    arr[jj * block_size : jj * block_size + len(example)-(p+2)] = example[:-(p+2)]
    arr[jj * block_size + len(example)-1 : (jj + 1) * block_size] = 0
    arr_label[jj * block_size : (1+jj) * block_size ] = -1
    arr_label[jj * block_size + (p+2)*C:jj * block_size + (p+2)*(C+1)] = example[-(p+2):]
    # pad to bock_size
arr.flush()
arr_label.flush()

writing train...


1000000it [00:12, 81239.66it/s]


In [21]:
print(arr[:200])

[-3  1  5  2  4  3 -4 -3  2  4  3  1  5 -4 -3  1  2  4  3  5 -4 -3  2  1
  5  4  3 -4 -3  2  1  3  4  5 -4 -3  5  1  4  2  3 -4 -3  1  3  2  4  5
 -4 -3  2  4  3  5  1 -4 -3  3  5  2  4  1 -4 -3  5  3  2  4  1 -4 -3  4
  2  3  1  5 -4 -3  5  3  4  1  2 -4 -3  3  2  4  5  1 -4 -3  3  2  1  5
  4 -4 -3  1  4  3  5  2 -4 -3  3  2  5  1  4 -4 -3  2  3  5  1  4 -4 -3
  3  1  5  2  4 -4 -2  4  5  3  1  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [5]:
data_dir = os.path.join('data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
train_label = np.memmap(os.path.join(data_dir, 'train_label.bin'), dtype=np.int16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
val_label = np.memmap(os.path.join(data_dir, 'val_label.bin'), dtype=np.int16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    label = train_label if split == 'train' else val_label
    ix = torch.randint(len(data) // block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i * block_size:(i+1)*block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((label[i * block_size:(i+1)*block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

In [5]:
random_ints = np.random.randint(low = 0,high =2, size = 100, dtype = np.int8)
exp = ''
for i in random_ints[:-1]:
    exp += str(i) + "+"
exp += str(i) + '='
print(exp)
random_ints.sum()

0+0+1+0+1+1+1+0+1+1+0+0+1+0+0+1+1+1+0+1+0+0+0+1+0+1+1+1+1+0+1+1+0+0+0+1+1+1+0+1+0+0+1+0+1+1+0+0+1+1+1+0+1+1+0+1+1+1+1+1+1+1+1+1+0+0+1+0+0+1+0+1+1+1+1+1+1+0+1+1+0+1+0+0+0+0+1+1+1+1+1+1+0+0+0+0+0+1+1+1=


59

In [7]:
random_ints

array([0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1], dtype=int8)

In [1]:
np.random.randn(100)

NameError: name 'np' is not defined